In [211]:
import matplotlib.pyplot as plt
import numpy as np

In [212]:
class Rule:
    def __init__(self, inputs, output):
        self.output = output
        self.inputs = inputs

    def evaluate(self, inputs):
        return [self.output, min([inputs[descriptorName][measureName] for descriptorName, measureName in self.inputs.items()])]

In [213]:
class Descriptor:
    def __init__(self):
        self.descriptors = {}
        self.inverse = {}

    def addMembership(self, name, membershipFunction, inverse=None):
        self.descriptors[name] = membershipFunction
        self.inverse[name] = inverse 

    def fuzzify(self, value):
        return {name: membershipFunction(value) for name, membershipFunction in self.descriptors.items()}

    def defuzzify(self, name, value):
        return self.inverse[name](value)

In [214]:
class FuzzySystem:
    def __init__(self, rules):
        self.inputDescriptors = {}
        self.outputDescriptor = None
        self.outputDescriptorName = None
        self.rules = rules

    def addInputDescriptor(self, name, descriptor):
        self.inputDescriptors[name] = descriptor
    
    def setOutputDescriptor(self, name, descriptor):
        self.outputDescriptor = descriptor
        self.outputDescriptorName = name

    def compute(self, inputData):
        fuzzyValues = self.__fuzzify(inputData)
        
        print("Fuzzy inputs:", fuzzyValues)
        
        ruleValues = self.__evaluateRules(fuzzyValues)
        fuzzyMeasures = [(list(descr[0].values())[0], descr[1]) for descr in ruleValues]
        
        print("Fuzzy output:", *fuzzyMeasures)
        
        weighted_total = 0
        weight_sum = 0
        for measure in fuzzyMeasures:
            weight_sum += measure[1]
            weighted_total += self.outputDescriptor.defuzzify(*measure) * measure[1]
        
        
        print("If temperature=" + str(inputData['temperature']) + " and capacity=" + str(inputData['capacity']) + " then power=" + str(weighted_total / weight_sum) + "\n")
        
        return weighted_total / weight_sum

    def __fuzzify(self, data):
        return {descriptorName: self.inputDescriptors[descriptorName].fuzzify(data[descriptorName]) for descriptorName, value in data.items()}

    def __evaluateRules(self, fuzzyValues):
        return [rule.evaluate(fuzzyValues) for rule in self.rules if rule.evaluate(fuzzyValues)[1] != 0]

In [215]:
class Controller:
    def __init__(self, temperature, capacity, power, rules):
        self.system = FuzzySystem(rules)
        self.system.addInputDescriptor('temperature', temperature)
        self.system.addInputDescriptor('capacity', capacity)
        self.system.setOutputDescriptor('power', power)

    def compute(self, inputs):
        return str(self.system.compute(inputs))

In [216]:
def membershipTrapezoidal(a, b, c, d):
    return lambda x: max(0, min((x-a)/(b-a), 1, (d-x)/(d-c)))

def membershipTrapezoidalNINF(c, d):
    return lambda x: max(0, min(1, (d-x)/(d-c)))

def membershipTrapezoidalINF(a, b):
    return lambda x: max(0, min((x-a)/(b-a), 1))

def inverseTrapezoidal(a, b):
    return lambda y: y*(b-a)+a

def membershipTriangular(a, b, c):
    return lambda x: max(0, min((x-a)/(b-a), 1, (c-x)/(c-b)))

def inverseTriangular(a, b, c):
    return lambda x: (inverseTrapezoidal(a, b)(x) + inverseTrapezoidal(c, b)(x)) / 2

In [217]:




temperature = Descriptor()
temperature.addMembership('cold', membershipTrapezoidalNINF(30, 50))
temperature.addMembership('cool', membershipTriangular(30, 50, 70))
temperature.addMembership('moderate', membershipTriangular(60, 70, 80))
temperature.addMembership('hot', membershipTriangular(70, 90, 110))
temperature.addMembership('very hot', membershipTrapezoidalINF(90, 110))

capacity = Descriptor()
capacity.addMembership('small', membershipTrapezoidalNINF(0, 5))
capacity.addMembership('medium', membershipTriangular(3, 5, 7))
capacity.addMembership('high', membershipTrapezoidalINF(5, 10))

power = Descriptor()
power.addMembership('small', membershipTrapezoidalNINF(0, 10), inverseTrapezoidal(10, 0))
power.addMembership('medium', membershipTriangular(5, 10, 15), inverseTriangular(5, 10, 15))
power.addMembership('high', membershipTrapezoidalINF(10, 20), inverseTrapezoidal(10, 20))


rules = []
rules.append(Rule({'temperature': 'cold', 'capacity': 'small'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'cool', 'capacity': 'small'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'moderate', 'capacity': 'small'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'hot', 'capacity': 'small'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'very hot', 'capacity': 'small'}, {'power': 'small'}))

rules.append(Rule({'temperature': 'cold', 'capacity': 'medium'}, {'power': 'medium'}))
rules.append(Rule({'temperature': 'cool', 'capacity': 'medium'}, {'power': 'medium'}))
rules.append(Rule({'temperature': 'moderate', 'capacity': 'medium'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'hot', 'capacity': 'medium'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'very hot', 'capacity': 'medium'}, {'power': 'small'}))

rules.append(Rule({'temperature': 'cold', 'capacity': 'high'}, {'power': 'high'}))
rules.append(Rule({'temperature': 'cool', 'capacity': 'high'}, {'power': 'high'}))
rules.append(Rule({'temperature': 'moderate', 'capacity': 'high'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'hot', 'capacity': 'high'}, {'power': 'small'}))
rules.append(Rule({'temperature': 'very hot', 'capacity': 'high'}, {'power': 'small'}))

ctrl = Controller(temperature, capacity, power, rules)

In [218]:
ctrl.compute({'temperature': 20, 'capacity': 10})
ctrl.compute({'temperature': 105, 'capacity': 1})
ctrl.compute({'temperature': 1, 'capacity': 5})
ctrl.compute({'temperature': 55, 'capacity': 4})
ctrl.compute({'temperature': 75, 'capacity': 9})

Fuzzy inputs: {'temperature': {'cold': 1, 'cool': 0, 'moderate': 0, 'hot': 0, 'very hot': 0}, 'capacity': {'small': 0, 'medium': 0, 'high': 1.0}}
Fuzzy output: ('high', 1)
If temperature=20 and capacity=10 then power=20.0

Fuzzy inputs: {'temperature': {'cold': 0, 'cool': 0, 'moderate': 0, 'hot': 0.25, 'very hot': 0.75}, 'capacity': {'small': 0.8, 'medium': 0, 'high': 0}}
Fuzzy output: ('small', 0.25) ('small', 0.75)
If temperature=105 and capacity=1 then power=3.75

Fuzzy inputs: {'temperature': {'cold': 1, 'cool': 0, 'moderate': 0, 'hot': 0, 'very hot': 0}, 'capacity': {'small': 0, 'medium': 1.0, 'high': 0}}
Fuzzy output: ('medium', 1)
If temperature=1 and capacity=5 then power=10.0

Fuzzy inputs: {'temperature': {'cold': 0, 'cool': 0.75, 'moderate': 0, 'hot': 0, 'very hot': 0}, 'capacity': {'small': 0.2, 'medium': 0.5, 'high': 0}}
Fuzzy output: ('small', 0.2) ('medium', 0.5)
If temperature=55 and capacity=4 then power=9.428571428571429

Fuzzy inputs: {'temperature': {'cold': 0, 'coo

'5.833333333333333'